# Download BioBERTpt para uso Offline

**Este notebook baixa o BioBERTpt e salva como output para uso offline.**

- Modelo: `pucpr/biobertpt-all`
- BERT pré-treinado em textos biomédicos em português
- Tamanho: ~440 MB

---
## ⚠️ IMPORTANTE: INTERNET DEVE ESTAR **ON**

**CONFIGURACAO KAGGLE:**
1. Settings → Internet → **ON** ← **OBRIGATÓRIO para download!**
2. Settings → Accelerator → **None** (não precisa GPU)
3. Execute **Run All**
4. Clique em **Save Version** → **Save & Run All**

**Se der erro de conexão:**
- Verifique se Internet está **ON** em Settings
- Tente novamente em alguns minutos (Kaggle pode ter instabilidade temporária)

**Depois de salvar:**
- O modelo fica disponível em **Add Input → Your Work → este notebook**
- Use em outros notebooks com Internet **OFF**

---

In [ ]:
# Instalar/atualizar transformers
!pip install -q transformers

# Verificar conectividade
import urllib.request
import socket

def check_internet():
    try:
        socket.setdefaulttimeout(10)
        urllib.request.urlopen('https://huggingface.co', timeout=10)
        return True
    except:
        return False

if check_internet():
    print("✅ Internet OK - conectado ao HuggingFace")
else:
    print("❌ ERRO: Sem conexão com internet!")
    print("   → Vá em Settings → Internet → ON")
    print("   → Reinicie a sessão e tente novamente")
    raise ConnectionError("Internet não está habilitada. Ative em Settings → Internet → ON")

In [ ]:
import os
from transformers import AutoTokenizer, AutoModel

# Modelo a baixar
MODEL_NAME = "pucpr/biobertpt-all"
OUTPUT_DIR = "/kaggle/working/biobertpt"

print(f"Baixando modelo: {MODEL_NAME}")
print(f"Destino: {OUTPUT_DIR}")

In [ ]:
# Baixar tokenizer e modelo (com retry)
import time

def download_with_retry(func, *args, max_retries=3, **kwargs):
    """Tenta baixar com retry em caso de falha de rede."""
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = (attempt + 1) * 10
                print(f"   ⚠️ Erro na tentativa {attempt + 1}: {str(e)[:100]}")
                print(f"   Aguardando {wait_time}s antes de tentar novamente...")
                time.sleep(wait_time)
            else:
                raise

print("[1/3] Baixando tokenizer...")
tokenizer = download_with_retry(AutoTokenizer.from_pretrained, MODEL_NAME)
print("   ✅ Tokenizer OK")

print("[2/3] Baixando modelo...")
model = download_with_retry(AutoModel.from_pretrained, MODEL_NAME)
print("   ✅ Modelo OK")

print("[3/3] Salvando em disco...")
os.makedirs(OUTPUT_DIR, exist_ok=True)
tokenizer.save_pretrained(OUTPUT_DIR)
model.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Modelo salvo em: {OUTPUT_DIR}")

In [ ]:
# Verificar arquivos salvos
print("Arquivos salvos:")
print("-" * 50)
total = 0
for f in sorted(os.listdir(OUTPUT_DIR)):
    path = os.path.join(OUTPUT_DIR, f)
    if os.path.isfile(path):
        size = os.path.getsize(path)
        total += size
        print(f"{f:40} {size/1024/1024:>8.2f} MB")
print("-" * 50)
print(f"TOTAL: {total/1024/1024:.2f} MB")

In [ ]:
# Testar carregamento local
print("Testando carregamento local...")
tok_test = AutoTokenizer.from_pretrained(OUTPUT_DIR)
mod_test = AutoModel.from_pretrained(OUTPUT_DIR)

# Teste rapido
text = "Mamografia sem alteracoes suspeitas. BI-RADS 2."
inputs = tok_test(text, return_tensors="pt")
outputs = mod_test(**inputs)

print("Teste OK!")
print(f"   Vocab: {tok_test.vocab_size}")
print(f"   Hidden: {mod_test.config.hidden_size}")
print(f"   Layers: {mod_test.config.num_hidden_layers}")

## Proximos Passos

1. Apos rodar, clique em **Save Version** -> **Save & Run All (Commit)**
2. Aguarde o notebook finalizar
3. O output ficara em: **Add Input -> Your Work -> este notebook**
4. Em outros notebooks, adicione como Input e use:

```python
MODEL_PATH = "/kaggle/input/download-biobertpt/biobertpt"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModel.from_pretrained(MODEL_PATH, local_files_only=True)
```